# Stabilities and accuracy in time. 

Lets consider same setup as in [ex_3a](https://github.com/AST-Course/AST5110/blob/main/ex_3a.ipynb) Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{1}$$ 

for the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right)\right]   \tag{2}$$

whereby $A = 0.02$ , $x_c = 0.70$, $W = 0.1$. Let the solution evolve until time $t_f = 100$. Explain in physical (or mathematical) terms the solution you get. However, let's now implement a new time-step method. 

Add the following method to your library and run the previous simulation. 

$$u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{v \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$

Apply von Neumann stability analysis to the expression above. The stability condition $|\xi(k)|^2 \leq 1$ leads to the famous Courant-Friedrichs-Lewy (CFL) stability. 

Add this to your library and solve the previous simulation imposing the CFL condition. For this exercise, fill in `nm_lib` the function `evolv_Lax_uadv_burgers`. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from importlib import reload

from nm_lib import nm_lib as nm, utils as utils
reload(nm);

plt.style.use("fast")

def get_uadv_t0(xx: np.ndarray, A: float = 0.02, W: float = 0.1, xc: float = 0.7, B: float = 0) -> np.ndarray:
    tanh_term = np.tanh((xx+xc)/W) - np.tanh((xx-xc)/W)
    return A*tanh_term + B

In [ ]:
if __name__ == "__main__":
    x0 = -1.4
    xf = 2.0
    nint = 256*2
    tf = 100

    xx, _ = utils.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0(xx)
    tt, uunt = nm.evolv_Lax_uadv_burgers(xx, uut0, ddx = nm.deriv_cent, bnd_limits=[1,1], end_time=tf, cfl_cut=0.98)
    print(tt[-1])

    anim = utils.animate_u(tt[::16], uunt[::16], xx, initial=uut0)
    display(HTML(anim.to_jshtml()))
    plt.close()

In [ ]:
if __name__ == "__main__":
    x0 = -1.4
    xf = 2.0
    nint = 256*2
    tf = 100

    xx, _ = utils.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0(xx)
    tt, uunt = nm.evolv_Lax_uadv_burgers(xx, uut0, ddx = nm.deriv_cent, bnd_limits=[1,1], end_time=tf, cfl_cut=0.3)
    print(tt[-1])

    anim = utils.animate_u(tt[::16], uunt[::16], xx, initial=uut0)
    display(HTML(anim.to_jshtml()))
    plt.close()

<span style="color:Orange">

### Von Neumann analysis

Solution on form $u_j^n = \xi^n e^{ikj\Delta x}$, require $\left|\xi(k)\right|<1$ for stability. Apply von Neumann on Lax–Friedrichs method:

We have

\begin{align*}
u_j^{n+1}   &= \xi^{n+1}e^{ijk\Delta x}\\
u_j^{n}     &= \xi^n e^{ijk\Delta x}\\
u_{j+1}^{n} &= \xi^n e^{ijk\Delta x}e^{ik\Delta x}\\
u_{j-1}^{n} &= \xi^n e^{ijk\Delta x}e^{-ik\Delta x}\\
\end{align*}

so

\begin{align}
\xi^{n+1}e^{ijk\Delta x} &= \frac{1}{2}\left(\xi^n e^{ijk\Delta x}e^{ik\Delta x} +  \xi^n e^{ijk\Delta x}e^{-ik\Delta x}\right) - \frac{v\Delta t}{2 \Delta x} \left(\xi^n e^{ijk\Delta x}e^{ik\Delta x} -  \xi^n e^{ijk\Delta x}e^{-ik\Delta x}\right) \\
&= \frac{1}{2}\xi^n e^{ijk\Delta x} \left(e^{ik\Delta x} + e^{-ik\Delta x} \right) - \frac{v\Delta t}{2 \Delta x}\xi^n e^{ijk\Delta x} \left(e^{ik\Delta x} - e^{-ik\Delta x}\right)
\quad \quad \big| \cdot \xi^{-n}e^{-ijk\Delta x} \\
\xi &= \frac{1}{2} \left(e^{ik\Delta x} + e^{-ik\Delta x} \right) - \frac{v\Delta t}{2 \Delta x}\left(e^{ik\Delta x} - e^{-ik\Delta x}\right)
\end{align}

using that $\cos(x) = \frac{e^{ix}+e^{-ix}}{2}$ and $\sin(x) = \frac{e^{ix}-e^{-ix}}{i2}$, and requiring $\left|\xi(k)\right|<1$ for all $k$ we have

\begin{align}
\xi &= \cos(k\Delta x) - \frac{v\Delta t}{2 \Delta x} i \sin(k\Delta x) \\
\left|\xi\right| &= \sqrt{\cos^2(k\Delta x) + \left(-\frac{v\Delta t}{2 \Delta x} \sin(k\Delta x)\right)^2 } <1
\end{align}

Max instability may occur for $k\Delta x = m \frac{\pi}{2}$ for $m\in \mathbb{Z}$, then $\cos^2(m\frac{\pi}{2}) = 0$ and $\sin^2(m\frac{\pi}{2}) =1$ so we must have $\left|\frac{v\Delta t}{\Delta x}\right| <1$ for $\left|\xi(k)\right|<1$ for all $k$, or $$\Delta t < \left|\frac{\Delta x}{v}\right|.$$

## 1- Diffusive. 

Redo the exercise [ex_2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb) and compare the two methods, i.e., one from 
[ex_2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb) and the Lax-Method. 

Which one is more diffusive? Why? Rewrite Lax-method, so the right-hand side is as follows: 

$$\frac{u^{n+1}-u^{n}}{\Delta t} = ...$$

What is the reminder term look like? Is the order of convergence the same for the two methods? 

---

(\*) Equation (1) is, in fact, a shortened version of the full Burgers’ equation, which contains a viscosity term on the right-hand side, as follows:
$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}.$$